In [2]:
import numpy as np
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11688 entries, 0 to 11687
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           11688 non-null  object 
 1   date                         11688 non-null  object 
 2   cluster_id                   11688 non-null  object 
 3   electricity_consumption      11688 non-null  float64
 4   temperature_2m_max           11688 non-null  float64
 5   temperature_2m_min           11688 non-null  float64
 6   apparent_temperature_max     11688 non-null  float64
 7   apparent_temperature_min     11688 non-null  float64
 8   sunshine_duration            11688 non-null  float64
 9   daylight_duration            11688 non-null  float64
 10  wind_speed_10m_max           11688 non-null  float64
 11  wind_gusts_10m_max           11688 non-null  float64
 12  wind_direction_10m_dominant  11688 non-null  float64
 13  shortwave_radiat

In [5]:
df.describe()

,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
count,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000
mean,642.313328,18.733385,7.174786,17.215315,4.784608,29302.656241,44070.779592,23.273058,46.420115,202.335554,12.742257,2.428585
std,287.673473,7.504161,5.960263,9.085303,7.260336,13600.113712,9283.839471,7.993389,15.226372,100.697741,7.588077,1.607845
min,207.472000,-1.900000,-10.800000,-8.100000,-15.400000,0.000000,29447.550000,6.800000,13.300000,0.002855,0.333333,0.173333
25%,414.376000,12.800000,2.500000,10.000000,-1.100000,19104.170833,35150.501667,17.400000,35.300000,134.384708,5.600000,0.990000
50%,602.672000,18.200000,7.100000,16.400000,4.400000,29917.360000,44185.056667,22.000000,43.900000,219.000000,12.140000,2.156667
75%,836.034500,24.300000,12.100000,24.000000,10.800000,40697.756667,52997.815833,28.000000,54.700000,281.609519,18.924167,3.616667
max,1658.348000,40.900000,23.200000,42.600000,23.800000,53498.910000,58501.420000,63.900000,126.700000,360.000000,29.983333,8.133333


In [6]:

le = LabelEncoder()

In [7]:
df['cluster_id'] = le.fit_transform(df['cluster_id'])
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek 
df['day_of_year'] = df['date'].dt.dayofyear
df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
df["quarter"] = df["date"].dt.quarter
df["is_weekend"] = df["date"].dt.dayofweek.isin([5, 6]).astype(int)

In [8]:
df.drop(columns=['ID', 'date'], inplace=True)


In [9]:
df['temp_range'] = df['temperature_2m_max'] - df['temperature_2m_min']
df['apparent_temp_range'] = df['apparent_temperature_max'] - df['apparent_temperature_min']

df['is_weekend_summer'] = (df['is_weekend'] == 1) & (df['month'].isin([6, 7, 8]))
# df['is_peak_month'] = df['month'].isin([6, 12])  # misalnya bulan panas/dingin

df['extreme_temp'] = (df['temperature_2m_max'] > 35) | (df['temperature_2m_min'] < 5)
# df['extreme_wind'] = df['wind_gusts_10m_max'] > 40

df['hot_weekend'] = (df['is_weekend'] == 1) & (df['temperature_2m_max'] > 32)


In [10]:
# Di training
# cluster_mean = df.groupby('cluster_id')['electricity_consumption'].mean().to_dict()


In [11]:
from scipy import stats

# Kolom numerik yang dicek
num_cols = [
    'electricity_consumption',
    'temperature_2m_max', 'temperature_2m_min',
    'apparent_temperature_max', 'apparent_temperature_min',
    'sunshine_duration', 'daylight_duration',
    'wind_speed_10m_max', 'wind_gusts_10m_max',
    'shortwave_radiation_sum', 'et0_fao_evapotranspiration'
]

# Hitung Z-score absolut
z_scores = np.abs(stats.zscore(df[num_cols]))

# Filter data yang semuanya Z-score < 3
df_clean = df[(z_scores < 3).all(axis=1)]

n_total = len(df)
n_clean = len(df_clean)
n_removed = n_total - n_clean
percentage_removed = (n_removed / n_total) * 100

print(f"Data awal       : {n_total} baris")
print(f"Data setelah bersih: {n_clean} baris")
print(f"Data terhapus   : {n_removed} baris ({percentage_removed:.2f}%)")



Data awal       : 11688 baris
Data setelah bersih: 11511 baris
Data terhapus   : 177 baris (1.51%)


In [12]:
df.head()

,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,...,day_of_week,day_of_year,week_of_year,quarter,is_weekend,temp_range,apparent_temp_range,is_weekend_summer,extreme_temp,hot_weekend
0,0,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,...,2,1,1,1,0,6.6,5.1,False,True,False
1,1,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,...,2,1,1,1,0,7.9,9.0,False,True,False
2,2,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,...,2,1,1,1,0,13.7,15.0,False,True,False
3,3,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,...,2,1,1,1,0,6.1,6.1,False,True,False
4,0,386.908,10.7,7.0,6.6,3.3,22372.000000,29850.226667,34.3,75.2,...,3,2,1,1,0,3.7,3.3,False,False,False


# MODELING

In [13]:
X = df.drop(columns=['electricity_consumption'])
y = df['electricity_consumption']

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [66]:
catboost = CatBoostRegressor(
    learning_rate=0.06303797370334635,
    depth=8,
    l2_leaf_reg=8.509949414892493e-07,
    bagging_temperature=0.09266008044520813,
    random_strength=0.5015478544179721,
    random_state=42
)
lightgbm = LGBMRegressor(
    colsample_bytree=0.6411585481655453,
    lambda_l1=0.000830811465288839,
    lambda_l2=0.0032360993995751053,
    learning_rate=0.04178605220965296,
    max_depth=17,
    min_child_samples=10,
    n_estimators=1025,
    num_leaves=34,
    subsample=0.6018222782970533,
    random_state=42
)
linear = LinearRegression()

In [67]:
estimators = [
    ('cat', catboost),
    ('lgbm', lightgbm)
    ]

stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(),  # Meta-model
    passthrough=True,  # Include original features to meta-model
    cv=5,  # Cross-validation
)

In [68]:
stacking_model.fit(Xtrain, ytrain)

0:	learn: 269.4531303	total: 6.31ms	remaining: 6.3s
1:	learn: 253.2919928	total: 9.28ms	remaining: 4.63s
2:	learn: 238.1772967	total: 11.8ms	remaining: 3.91s
3:	learn: 223.9929466	total: 14ms	remaining: 3.47s
4:	learn: 210.7197385	total: 16.6ms	remaining: 3.31s
5:	learn: 198.4167293	total: 19.8ms	remaining: 3.29s
6:	learn: 186.8936210	total: 22.9ms	remaining: 3.25s
7:	learn: 175.9701315	total: 26.8ms	remaining: 3.32s
8:	learn: 165.6810560	total: 29.5ms	remaining: 3.24s
9:	learn: 156.0907362	total: 32.1ms	remaining: 3.18s
10:	learn: 147.1054404	total: 35.1ms	remaining: 3.15s
11:	learn: 138.7602373	total: 37.9ms	remaining: 3.12s
12:	learn: 131.0370646	total: 40.4ms	remaining: 3.07s
13:	learn: 123.7537583	total: 43.1ms	remaining: 3.04s
14:	learn: 117.0204296	total: 45.3ms	remaining: 2.98s
15:	learn: 110.7304446	total: 47.4ms	remaining: 2.91s
16:	learn: 104.8628254	total: 50.8ms	remaining: 2.94s
17:	learn: 99.2895176	total: 52.9ms	remaining: 2.88s
18:	learn: 94.1204460	total: 55.1ms	remain

StackingRegressor(cv=5,
                  estimators=[('cat',
                               <catboost.core.CatBoostRegressor object at 0x176f3d3d0>),
                              ('lgbm',
                               LGBMRegressor(colsample_bytree=0.6411585481655453,
                                             lambda_l1=0.000830811465288839,
                                             lambda_l2=0.0032360993995751053,
                                             learning_rate=0.04178605220965296,
                                             max_depth=17, min_child_samples=10,
                                             n_estimators=1025, num_leaves=34,
                                             random_state=42,
                                             subsample=0.6018222782970533))],
                  final_estimator=Ridge(), passthrough=True)

In [69]:
ypred = stacking_model.predict(Xtest)

[LightGBM] [Warning] lambda_l1 is set=0.000830811465288839, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.000830811465288839
[LightGBM] [Warning] lambda_l2 is set=0.0032360993995751053, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0032360993995751053


In [70]:
rmse = root_mean_squared_error(ytest, ypred)
print(f'Root Mean Squared Error: {rmse:.2f}')

Root Mean Squared Error: 18.75


In [47]:
persentase = (rmse / np.mean(ytest)) * 100
print(f'Percentage of RMSE: {persentase:.2f}%')

Percentage of RMSE: 2.90%


In [53]:
df_tes = pd.read_csv('test.csv')

In [54]:
df_tes['cluster_id'] = le.fit_transform(df_tes['cluster_id'])
df_tes['date'] = pd.to_datetime(df_tes['date'])
df_tes['year'] = df_tes['date'].dt.year
df_tes['month'] = df_tes['date'].dt.month
df_tes['day'] = df_tes['date'].dt.day
df_tes['day_of_week'] = df_tes['date'].dt.dayofweek 
df_tes['day_of_year'] = df_tes['date'].dt.dayofyear
df_tes['week_of_year'] = df_tes['date'].dt.isocalendar().week.astype(int)
df_tes["quarter"] = df_tes["date"].dt.quarter
df_tes["is_weekend"] = df_tes["date"].dt.dayofweek.isin([5, 6]).astype(int)

In [55]:
df_tes.drop(columns=['ID', 'date'], inplace=True)

In [56]:
df_tes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cluster_id                   3400 non-null   int64  
 1   temperature_2m_max           3400 non-null   float64
 2   temperature_2m_min           3400 non-null   float64
 3   apparent_temperature_max     3400 non-null   float64
 4   apparent_temperature_min     3400 non-null   float64
 5   sunshine_duration            3400 non-null   float64
 6   daylight_duration            3400 non-null   float64
 7   wind_speed_10m_max           3400 non-null   float64
 8   wind_gusts_10m_max           3400 non-null   float64
 9   wind_direction_10m_dominant  3400 non-null   float64
 10  shortwave_radiation_sum      3400 non-null   float64
 11  et0_fao_evapotranspiration   3400 non-null   float64
 12  year                         3400 non-null   int32  
 13  month             

In [57]:
df_tes['temp_range'] = df_tes['temperature_2m_max'] - df_tes['temperature_2m_min']
df_tes['apparent_temp_range'] = df_tes['apparent_temperature_max'] - df_tes['apparent_temperature_min']

df_tes['is_weekend_summer'] = (df_tes['is_weekend'] == 1) & (df_tes['month'].isin([6, 7, 8]))
# df_tes['is_peak_month'] = df_tes['month'].isin([6, 12])  # misalnya bulan panas/dingin

df_tes['extreme_temp'] = (df_tes['temperature_2m_max'] > 35) | (df_tes['temperature_2m_min'] < 5)
# df_tes['extreme_wind'] = df_tes['wind_gusts_10m_max'] > 40

df_tes['hot_weekend'] = (df_tes['is_weekend'] == 1) & (df_tes['temperature_2m_max'] > 32)

In [58]:
rmse = root_mean_squared_error(ytest, ypred)
print(f'Root Mean Squared Error: {rmse:.2f}')

Root Mean Squared Error: 18.76


In [59]:
df_tes.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cluster_id                   3400 non-null   int64  
 1   temperature_2m_max           3400 non-null   float64
 2   temperature_2m_min           3400 non-null   float64
 3   apparent_temperature_max     3400 non-null   float64
 4   apparent_temperature_min     3400 non-null   float64
 5   sunshine_duration            3400 non-null   float64
 6   daylight_duration            3400 non-null   float64
 7   wind_speed_10m_max           3400 non-null   float64
 8   wind_gusts_10m_max           3400 non-null   float64
 9   wind_direction_10m_dominant  3400 non-null   float64
 10  shortwave_radiation_sum      3400 non-null   float64
 11  et0_fao_evapotranspiration   3400 non-null   float64
 12  year                         3400 non-null   int32  
 13  month             

In [60]:
hasil = stacking_model.predict(df_tes)

[LightGBM] [Warning] lambda_l1 is set=1.702637694606324e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.702637694606324e-05
[LightGBM] [Warning] lambda_l2 is set=0.006027551301719578, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.006027551301719578


In [61]:
df_hasil = pd.read_csv('test.csv')

In [62]:
df_hasil.head()

,ID,date,cluster_id,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,cluster_1_2022-01-01,2022-01-01,cluster_1,15.2,7.9,14.2,5.2,21902.986667,29784.506667,22.2,37.8,172.732245,4.173333,0.780000
1,cluster_2_2022-01-01,2022-01-01,cluster_2,15.9,4.1,15.3,1.3,23550.080000,30647.540000,22.7,38.5,163.972679,5.146667,0.730000
2,cluster_3_2022-01-01,2022-01-01,cluster_3,18.6,4.9,18.8,1.8,24179.643333,31545.103333,15.5,26.3,198.645137,5.713333,0.730000
3,cluster_4_2022-01-01,2022-01-01,cluster_4,13.3,3.9,11.9,1.3,23839.203333,30766.446667,16.1,27.4,183.517316,4.200000,0.486667
4,cluster_1_2022-01-02,2022-01-02,cluster_1,14.1,8.1,11.2,5.3,21018.193333,29847.110000,34.4,59.4,223.335840,4.030000,1.170000


In [63]:
df_hasil['electricity_consumption'] = hasil

In [64]:
df_hasil.head()

,ID,date,cluster_id,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,electricity_consumption
0,cluster_1_2022-01-01,2022-01-01,cluster_1,15.2,7.9,14.2,5.2,21902.986667,29784.506667,22.2,37.8,172.732245,4.173333,0.780000,316.602773
1,cluster_2_2022-01-01,2022-01-01,cluster_2,15.9,4.1,15.3,1.3,23550.080000,30647.540000,22.7,38.5,163.972679,5.146667,0.730000,480.936956
2,cluster_3_2022-01-01,2022-01-01,cluster_3,18.6,4.9,18.8,1.8,24179.643333,31545.103333,15.5,26.3,198.645137,5.713333,0.730000,656.708950
3,cluster_4_2022-01-01,2022-01-01,cluster_4,13.3,3.9,11.9,1.3,23839.203333,30766.446667,16.1,27.4,183.517316,4.200000,0.486667,952.449469
4,cluster_1_2022-01-02,2022-01-02,cluster_1,14.1,8.1,11.2,5.3,21018.193333,29847.110000,34.4,59.4,223.335840,4.030000,1.170000,354.623610


In [65]:
df_hasil[['ID', 'electricity_consumption']].to_csv('Data Science Academy Submission(18.76).csv', index=False)